##### References
* https://youtu.be/R8KB-Zcynxc?si=0LNzv1q7DugMXUCR
* https://youtu.be/ny215UUXbhI?si=TtY8tMmZF9CmGnK2


## Building the simplest Graph

하나의 엣지(edge)로 연결된 두 개의 노드(node)가 있는 그래프로 시작합니다.

In [5]:
#!pip install langgraph

Defaulting to user installation because normal site-packages is not writeable
  Obtaining dependency information for langgraph from https://files.pythonhosted.org/packages/bc/6f/ad63ff73f5baade85bf5ebcc190301ab7725e5b60b469859213456f3e730/langgraph-0.1.1-py3-none-any.whl.metadata
  Obtaining dependency information for langchain-core<0.3,>=0.2 from https://files.pythonhosted.org/packages/cd/79/319f4898ce837e62d3611ed5da9246d10e3a966bb60fadb54901563867cf/langchain_core-0.2.10-py3-none-any.whl.metadata
  Obtaining dependency information for langsmith<0.2.0,>=0.1.75 from https://files.pythonhosted.org/packages/11/4f/1d89e18d542b4545787a806dd8b485b5178781cd6c99e7c79285fd862a34/langsmith-0.1.82-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/87.3 kB ? eta -:--:--
   ---------------------------------------- 87.3/87.3 kB ? eta 0:00:00
   ---------------------------------------- 0.0/332.8 kB ? eta -:--:--
   --------------------------------------- 332.8/332.8 kB 20.2 M

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain 0.1.16 requires langchain-core<0.2.0,>=0.1.42, but you have langchain-core 0.2.10 which is incompatible.
langchain-community 0.0.34 requires langchain-core<0.2.0,>=0.1.45, but you have langchain-core 0.2.10 which is incompatible.
langchain-text-splitters 0.0.1 requires langchain-core<0.2.0,>=0.1.28, but you have langchain-core 0.2.10 which is incompatible.


In [2]:
# from Ipython.display import Image
# Image(url="https://pbs.twimg.com/media/GGcD9z2W4AAeSHE?format=jpg&name=small", width=400)

<img src="https://pbs.twimg.com/media/GGcD9z2W4AAeSHE?format=jpg&name=small" width="400">

노드는 필요에 따라 호출할 수 있는 함수처럼 작동합니다. 우리의 경우 node_1이 시작점이고 node_2가 종료점입니다.
<img src="langgraph-2.png" width="400">

In [6]:
def function_1(input_1):
    return input_1 + " Hi "

def function_2(input_2):
    return input_2 + "there"

In [7]:
from langgraph.graph import Graph

workflow = Graph()

workflow.add_node("node_1", function_1)
workflow.add_node("node_2", function_2)
workflow.add_edge('node_1', 'node_2')
workflow.set_entry_point("node_1")
workflow.set_finish_point("node_2")

app = workflow.compile()

In [11]:
app.invoke("Hello")

'Hello Hi there'

In [27]:
input='Hello'
for output in app.stream(input):
    # stream() yields dictionaries with output keyed by node name
    for key, value in output.items():
        print(f"Output from node '{key}': '{value}'")
    print("======")

Output from node 'agent': 'Hello! How can I assist you today?'
Output from node 'node_2': 'Agent Says: Hello! How can I assist you today?'


보시다시피 노드를 함수처럼 실행하고 그로부터 일부 값을 반환할 수 있습니다.

### Adding LLM Call

이제 첫 번째 노드를 Open AI model을 호출할 수 있는 "agent"로 만들어 보겠습니다. 우리는 langchain을 사용하여 이 호출을 쉽게 만들 수 있습니다.

<img src="langgraph-3.png" width="400">

In [18]:
#!pip install langchain langchain_openai
#!pip install python-dotenv

Defaulting to user installation because normal site-packages is not writeable
  Obtaining dependency information for langchain_openai from https://files.pythonhosted.org/packages/8c/a7/79f6c6495d6eeb88d4c435cba4fe2e4da5a3d5f608793dc4753033ef2c73/langchain_openai-0.1.10-py3-none-any.whl.metadata
  Obtaining dependency information for langchain-core<0.2.0,>=0.1.42 from https://files.pythonhosted.org/packages/43/8b/48b7e6de9041d2b33d5108e154b82d1bd6c47cc68f0e44cb4fcdaccf5ec7/langchain_core-0.1.52-py3-none-any.whl.metadata
INFO: pip is looking at multiple versions of langchain-openai to determine which version is compatible with other requirements. This could take a while.
  Obtaining dependency information for langchain_openai from https://files.pythonhosted.org/packages/15/bb/e8f080c8408673609f15436237ec8f9d8c39ab67986807d0ca2663acd7a0/langchain_openai-0.1.9-py3-none-any.whl.metadata
  Obtaining dependency information for langchain_openai from https://files.pythonhosted.org/packages/95/a

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langgraph 0.1.1 requires langchain-core<0.3,>=0.2, but you have langchain-core 0.1.52 which is incompatible.


LangChain에서 ChatOpenAI 모델에 대한 일반적인 호출은 다음과 같이 수행됩니다.

먼저 OpenAI용 API 키를 설정합니다.

In [19]:
from dotenv import load_dotenv
import os

load_dotenv()
os.environ['OPENAI_API_KEY'] = os.environ.get("OPENAI_API_KEY")

In [20]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(temperature=0)
model.invoke('Hey there')

AIMessage(content='Hello! How can I assist you today?', response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 9, 'total_tokens': 18}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-0b83c275-b924-4f4a-851a-9d68aef226dd-0')

AI 응답만 보고 싶다면 다음을 수행할 수 있습니다.

In [21]:
model.invoke('Hey there').content

'Hello! How can I assist you today?'

좋습니다! 이를 염두에 두고 사용자 질문을 모델에 보낼 수 있도록 위의 function_1을 변경해 보겠습니다. 그런 다음 이 응답을 function_2로 보내면 짧은 문자열이 추가되어 사용자에게 반환됩니다.

In [23]:
def function_1(input_1):
    response = model.invoke(input_1)
    return response.content

def function_2(input_2):
    return "Agent Says: " + input_2

In [24]:
workflow = Graph()
workflow.add_node("agent", function_1)
workflow.add_node("node_2", function_2)
workflow.add_edge('agent', 'node_2')
workflow.set_entry_point("agent")
workflow.set_finish_point("node_2")
app=workflow.compile()

In [25]:
app.invoke("Hey there")

'Agent Says: Hello! How can I assist you today?'

In [26]:
input = 'Hey there'
for output in app.stream(input):
    for key, value in output.items():
        print(f"Output from node '{key}': '{value}'")
    print("====")

Output from node 'agent': 'Hello! How can I assist you today?'
====
Output from node 'node_2': 'Agent Says: Hello! How can I assist you today?'
====


### First functional Agent App - City Temperature

<img src="langgraph-4.png" width="400">

#### Step 1: Parse the city mentioned

사용자가 query에서 언급한 도시를 추출해 보겠습니다.


In [28]:
def function_1(input_1):
    complete_query = "Your task is to provide only the city name based on the user query. \
        Nothing more, just the city name mentioned. Following is the user query: " + input_1
    response = model.invoke(complete_query)
    return response.content

def function_2(input_2):
    return "Agent Says: " + input_2

In [29]:
# Define a Langchain graph
workflow = Graph()

#calling node 1 as agent
workflow.add_node("agent", function_1)
workflow.add_node("node_2", function_2)

workflow.add_edge('agent', 'node_2')

workflow.set_entry_point("agent")
workflow.set_finish_point("node_2")

app = workflow.compile()

In [30]:
app.invoke("What's the temperature in Las Vegas")

'Agent Says: Las Vegas'

#### Step 2: Adding a weather API call
function_2가 도시 이름을 가져와 해당 도시의 날씨를 제공하도록 하려면 어떻게 해야 할까요?

우리는 Open Weather Map이 LangChain에 통합되어 있다는 것을 알고 있습니다.

pyown 을 설치하고 [Open Weather Map 웹사이트](https://openweathermap.org/) 에서 API 키를 생성한 다음, 아래 셀을 실행하여 특정 도시의 날씨를 가져와야 합니다.

In [31]:
#!pip install pyowm

Defaulting to user installation because normal site-packages is not writeable
  Obtaining dependency information for pyowm from https://files.pythonhosted.org/packages/6e/88/1279817aa7f5988a2ff42a6755fd371f3c1806aca377cb63b3d16684b174/pyowm-3.3.0-py3-none-any.whl.metadata
  Obtaining dependency information for geojson<3,>=2.3.0 from https://files.pythonhosted.org/packages/e4/8d/9e28e9af95739e6d2d2f8d4bef0b3432da40b7c3588fbad4298c1be09e48/geojson-2.5.0-py2.py3-none-any.whl.metadata
   ---------------------------------------- 0.0/4.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/4.5 MB ? eta -:--:--
    --------------------------------------- 0.1/4.5 MB 919.0 kB/s eta 0:00:05
   -- ------------------------------------- 0.3/4.5 MB 2.4 MB/s eta 0:00:02
   ----- ---------------------------------- 0.6/4.5 MB 3.7 MB/s eta 0:00:02
   ------ --------------------------------- 0.7/4.5 MB 3.8 MB/s eta 0:00:02
   ------ --------------------------------- 0.7/4.5 MB 3.8 MB/s eta 0:

In [32]:
from langchain_community.utilities import OpenWeatherMapAPIWrapper
load_dotenv()
os.environ["openweathermap_api_key"] = os.environ.get("OPENWEATHERMAP_API_KEY")

weather = OpenWeatherMapAPIWrapper()

In [33]:
weather_data = weather.run("Las Vegas")
print(weather_data)

In Las Vegas, the current weather is as follows:
Detailed status: clear sky
Wind speed: 6.69 m/s, direction: 170°
Humidity: 23%
Temperature: 
  - Current: 37.95°C
  - High: 39.69°C
  - Low: 36.01°C
  - Feels like: 37.3°C
Rain: {}
Heat index: None
Cloud cover: 0%


이제 이를 function_2에 통합하고 workflow에서 "node_2" 대신 "tool" 또는 "weather_agent"로 function_2를 호출해 보겠습니다.

In [34]:
def function_1(input_1):
    complete_query = "Your task is to provide only the city name based on the user query. \
    Nothing more, just the city name mentioned. Following is the user query: " + input_1
    response = model.invoke(complete_query)
    return response.content

def function_2(input_2):
    weather_data = weather.run(input_2)
    return weather_data

In [35]:
from langgraph.graph import Graph

workflow = Graph()
workflow.add_node('agent', function_1)
workflow.add_node('tool', function_2)
workflow.add_edge('agent', 'tool')
workflow.set_entry_point('agent')
workflow.set_finish_point('tool')
app = workflow.compile()

In [36]:
app.invoke("What's the temperature in Las Vegas")

'In Las Vegas, the current weather is as follows:\nDetailed status: clear sky\nWind speed: 6.69 m/s, direction: 170°\nHumidity: 23%\nTemperature: \n  - Current: 37.95°C\n  - High: 39.69°C\n  - Low: 36.01°C\n  - Feels like: 37.3°C\nRain: {}\nHeat index: None\nCloud cover: 0%'

In [37]:
input = "What's the temperature in Las Vegas"
for output in app.stream(input):
    for key, value in output.items():
        print(f"Output from node '{key}': '{value}'")
    print('====')

Output from node 'agent': 'Las Vegas'
====
Output from node 'tool': 'In Las Vegas, the current weather is as follows:
Detailed status: clear sky
Wind speed: 6.69 m/s, direction: 170°
Humidity: 23%
Temperature: 
  - Current: 37.95°C
  - High: 39.69°C
  - Low: 36.01°C
  - Feels like: 37.3°C
Rain: {}
Heat index: None
Cloud cover: 0%'
====


#### Step 3 Adding another LLM Call to filter results
온도만 원한다면 어떻게 해야할까요? 현재 설정에서는 전체 일기 예보를 제공합니다.

데이터를 필터링하기 위해 또 다른 LLM 호출을 할 수 있습니다.

<img src="langgraph-5.png" width="400">

In [38]:
def function_3(input_3):
    complete_query = "Your task is to provide info concisely based on the user query. Following is the user query: " + "user input"
    response = model.invoke(complete_query)
    return response.content

그러나 문제는 node_2에서 "user_input"을 알 수 없다는 것입니다.

첫 번째 node 에서 마지막 node 까지 "user_input"을 전달할 수 있나요?

예, dictionary 을 사용하여 node 간에 전달할 수 있습니다(list 만 사용할 수도 있지만, dictionary 를 사용하면 좀 더 쉽습니다).

In [41]:
# assign AgentState as an empty dict
AgentState = {}

# messages key will be assigned as an empty array. We will append new messages as we pass along nodes. 
AgentState["messages"] = []

In [42]:
AgentState

{'messages': []}

우리의 목표는 이 상태(State)를 다음과 같이 채우는 것입니다: 
```
{'messages': [HumanMessage, AIMessage, ...]]}
```
또한 이제 새 AgentState 에 따라 정보를 전달하도록 함수를 수정해야 합니다.

In [43]:
def function_1(state):
    messages = state['messages']
    user_input = messages[-1]
    complete_query = "Your task is to provide only the city name based on the user query. \
                    Nothing more, just the city name mentioned. Following is the user query: " + user_input
    response = model.invoke(complete_query)
    state['messages'].append(response.content) # appending AIMessage response to the AgentState
    return state

def function_2(state):
    messages = state['messages']
    agent_response = messages[-1]
    weather = OpenWeatherMapAPIWrapper()
    weather_data = weather.run(agent_response)
    state['messages'].append(weather_data)
    return state

def function_3(state):
    messages = state['messages']
    user_input = messages[0]
    available_info = messages[-1]
    agent2_query = "Your task is to provide info concisely based on the user query and the available information from the internet. \
                        Following is the user query: " + user_input + " Available information: " + available_info
    response = model.invoke(agent2_query)
    return response.content

In [44]:
from langgraph.graph import Graph

workflow = Graph()

workflow.add_node("agent", function_1)
workflow.add_node("tool", function_2)
workflow.add_node("responder", function_3)

workflow.add_edge('agent', 'tool')
workflow.add_edge('tool', 'responder')

workflow.set_entry_point("agent")
workflow.set_finish_point("responder")

app = workflow.compile()

In [45]:
inputs = {"messages": ["what is the temperature in Las Vegas"]}
app.invoke(inputs)

'The current temperature in Las Vegas is 36.7°C with clear skies and a low humidity of 27%.'

In [46]:
input = {"messages": ["what is the temperature in  Las Vegas"]}
for output in app.stream(input):
    for key, value in output.items():
        print(f"Output from node '{key}': '{value}'")
    print("====")

Output from node 'agent': '{'messages': ['what is the temperature in  Las Vegas', 'Las Vegas']}'
====
Output from node 'tool': '{'messages': ['what is the temperature in  Las Vegas', 'Las Vegas', 'In Las Vegas, the current weather is as follows:\nDetailed status: clear sky\nWind speed: 1.34 m/s, direction: 267°\nHumidity: 27%\nTemperature: \n  - Current: 36.7°C\n  - High: 37.28°C\n  - Low: 36.18°C\n  - Feels like: 36.48°C\nRain: {}\nHeat index: None\nCloud cover: 8%']}'
====
Output from node 'responder': 'The current temperature in Las Vegas is 36.7°C with clear skies, a low humidity of 27%, and a light wind speed of 1.34 m/s.'
====


array 에 많은 덧붙임(appending)이 진행되고 있음을 알 수 있으므로, 다음을 사용하여 좀 더 쉽게 만들 수 있습니다:

```python
from typing import TypedDict, Annotated, Sequence
import operator
from langchain_core.messages import BaseMessage

class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]
```
기본적으로 이전에 본 것처럼 State dictionary 를 만들고, 다음을 수행할 때 새 메시지가 메시지 배열에 추가되는지 확인합니다.

```python
{"messages": [new_array_element]}
```

또한 우리 앱이 "how are you?"와 같은 간단한 질문에 답할 수 없다는 것도 알고 있습니다.

In [47]:
inputs = {"messages": ["How are you?"]}
app.invoke(inputs)

"I am an AI assistant, so I don't have feelings, but thank you for asking. In Istanbul, the current weather is clear with a temperature of 22.89°C, a wind speed of 7.72 m/s, and 61% humidity."

이는 우리가 항상 도시를 분석한 다음 날씨를 찾고 싶기 때문입니다.

사용자에게 단순히 응답하는 것이 아니라, 필요한 경우에만 도구를 사용하도록 하여 에이전트를 더 똑똑하게 만들 수 있습니다.

LangGraph를 수행할 수 있는 방법은 다음과 같습니다.

1. tool 를 agent 에 바인딩
2. tool 를 호출할지 여부를 선택할 수 있는 옵션으로 agent에 conditianal edge 를 추가합니다.
3. tool 를 호출할 시기와 같이 conditianal edge에 대한 기준을 정의합니다. 이에 대한 함수를 정의하겠습니다.

위의 몇 가지 셀에서 언급한 AgentState 정의부터 시작해 보겠습니다.

In [48]:
from typing import TypedDict, Annotated, Sequence
import operator
from langchain_core.messages import BaseMessage

class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]

tool을 에이전트(LLM 모델)에 바인딩하는 것이 langchain에서 쉽게 만들어집니다.

In [49]:
from langchain_core.utils.function_calling import convert_to_openai_function
from langchain_community.tools.openweathermap import OpenWeatherMapQueryRun
from langchain_core.utils.function_calling import convert_to_openai_function

tools = [OpenWeatherMapQueryRun()]

model = ChatOpenAI(temperature=0, streaming=True)
functions = [convert_to_openai_function(t) for t in tools]
model = model.bind_functions(functions)

수정된 function_1은 이제 아래와 같습니다. 그 이유는 인간의 메시지를 state로 전달하고, 응답을 state에 추가하기 때문입니다. 또한 이제 agent 에는 사용할 수 있는 tool 이 바인딩되어 있습니다.

In [56]:
def function_1(state):
    messages = state['messages']
    response = model.invoke(messages)
    return {"messages": [response]}

function_2의 경우 tool 을 설정하고 호출하기를 원합니다. ToolInvocation을 사용하고, ToolExecuter로 실행하면 LangChain 에서 tool 을 쉽게 호출할 수 있습니다. 그런 다음 agent(node_1)가 tool 이 사용되었고, tool 의 응답이 가용하다는 것을 알 수 있도록, FunctionMessage 로 다시 응답합니다.

In [57]:
from langgraph.prebuilt import ToolInvocation
import json
from langchain_core.messages import FunctionMessage
from langgraph.prebuilt import ToolExecutor

tool_executor = ToolExecutor(tools)

def function_2(state):
    messages = state['messages']
    last_message = messages[-1] # this has the query we need to send to the tool provided by the agent

    parsed_tool_input = json.loads(last_message.additional_kwargs["function_call"]["arguments"])

    # We construct an ToolInvocation from the function_call and pass in the tool name and the expected str input for OpenWeatherMap tool
    action = ToolInvocation(
        tool=last_message.additional_kwargs["function_call"]["name"],
        tool_input=parsed_tool_input['__arg1'],
    )
    
    # We call the tool_executor and get back a response
    response = tool_executor.invoke(action)

    # We use the response to create a FunctionMessage
    function_message = FunctionMessage(content=str(response), name=action.tool)

    # We return a list, because this will get added to the existing list
    return {"messages": [function_message]}

마지막으로, 어떤 방향(tool 또는 사용자 응답)으로 가야 할지 파악하는 데 도움이 되는 conditional edge 에 대한 함수를 정의합니다.

우리는 tool 이름으로 function_call을 만들기 위한 additional_kwargs 를 가지고 있는 LangChain의 agent(LLM) 응답으로부터 이점을 얻을 수 있습니다.

따라서 우리의 논리는 additional_kwargs 에서 function_call을 사용할 수 있으면 tool 을 호출하고, 그렇지 않으면 논의를 종료하고 사용자에게 다시 응답하는 것입니다.

<img src="langgraph-6.png" width="600">

In [58]:
def where_to_go(state):
    messages = state['messages']
    last_message = messages[-1]
    
    if "function_call" in last_message.additional_kwargs:
        return "continue"
    else:
        return "end"

이제 위의 모든 변경 사항을 적용하여 LangGraph 앱이 아래와 같이 수정되었습니다.

In [59]:
# from langgraph.graph import Graph, END

# workflow = Graph()

# Or you could import StateGraph and pass AgentState to it
from langgraph.graph import StateGraph, END
workflow = StateGraph(AgentState)

workflow.add_node("agent", function_1)
workflow.add_node("tool", function_2)

# The conditional edge requires the following info below.
# First, we define the start node. We use `agent`.
# This means these are the edges taken after the `agent` node is called.
# Next, we pass in the function that will determine which node is called next, in our case where_to_go().

workflow.add_conditional_edges("agent", where_to_go,{   # Based on the return from where_to_go
                                                        # If return is "continue" then we call the tool node.
                                                        "continue": "tool",
                                                        # Otherwise we finish. END is a special node marking that the graph should finish.
                                                        "end": END
                                                    }
)

# We now add a normal edge from `tools` to `agent`.
# This means that if `tool` is called, then it has to call the 'agent' next. 
workflow.add_edge('tool', 'agent')

# Basically, agent node has the option to call a tool node based on a condition, 
# whereas tool node must call the agent in all cases based on this setup.

workflow.set_entry_point("agent")


app = workflow.compile()

또한 langchain에서 사용할 수 있는 HumanMessage 구성 요소를 사용하여 첫 번째 메시지를 전달하므로 AIMessage 및 FunctionMessage와 쉽게 구별할 수 있습니다.

In [60]:
from langchain_core.messages import HumanMessage

inputs = {"messages": [HumanMessage(content="what is the temperature in las vegas")]}
app.invoke(inputs)

{'messages': [HumanMessage(content='what is the temperature in las vegas'),
  AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"__arg1":"Las Vegas"}', 'name': 'open_weather_map'}}, response_metadata={'finish_reason': 'function_call'}, id='run-f0a8b6c2-4c14-4740-8f82-13656a0677e4-0'),
  FunctionMessage(content='In Las Vegas, the current weather is as follows:\nDetailed status: clear sky\nWind speed: 4.63 m/s, direction: 180°\nHumidity: 24%\nTemperature: \n  - Current: 37.24°C\n  - High: 38.38°C\n  - Low: 36.01°C\n  - Feels like: 36.54°C\nRain: {}\nHeat index: None\nCloud cover: 0%', name='open_weather_map'),
  AIMessage(content='The current temperature in Las Vegas is 37.24°C.', response_metadata={'finish_reason': 'stop'}, id='run-57f56fb8-303f-4ce7-a169-669f732d643b-0')]}

In [61]:
inputs = {"messages": [HumanMessage(content="what is the temperature in las vegas")]}
for output in app.stream(inputs):
    # stream() yields dictionaries with output keyed by node name
    for key, value in output.items():
        print(f"Output from node '{key}': '{value}'")
    print("=======")

Output from node 'agent': '{'messages': [AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"__arg1":"Las Vegas"}', 'name': 'open_weather_map'}}, response_metadata={'finish_reason': 'function_call'}, id='run-4c894e29-d3a5-4d4b-828e-635762622960-0')]}'
Output from node 'tool': '{'messages': [FunctionMessage(content='In Las Vegas, the current weather is as follows:\nDetailed status: clear sky\nWind speed: 4.63 m/s, direction: 180°\nHumidity: 24%\nTemperature: \n  - Current: 37.24°C\n  - High: 38.38°C\n  - Low: 36.01°C\n  - Feels like: 36.54°C\nRain: {}\nHeat index: None\nCloud cover: 0%', name='open_weather_map')]}'
Output from node 'agent': '{'messages': [AIMessage(content='The current temperature in Las Vegas is 37.24°C.', response_metadata={'finish_reason': 'stop'}, id='run-eada6116-159d-4c33-b304-f5f046aa91df-0')]}'


In [62]:
inputs = {"messages": [HumanMessage(content="how are you?")]}
app.invoke(inputs)['messages'][-1].content

"I'm just a computer program, so I don't have feelings, but I'm here to help you with any questions or tasks you have. How can I assist you today?"

In [63]:
inputs = {"messages": [HumanMessage(content="what is the temperature in Seoul")]}
app.invoke(inputs)['messages'][-1].content

'The current temperature in Seoul is 30.74°C.'

이를 통해 우리가 LangGraph 앱을 구축한 방법과 다양한 LangChain 구성요소를 사용한 이유를 잘 이해할 수 있기를 바랍니다.